<a href="https://colab.research.google.com/github/christievanderuit/TM10007_group11/blob/Caitlyn/TM10007_Group11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sklearn numpy matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2955 sha256=cd1602b6d3eaeec227351044d6f45940c230dcbf6424b37e97f5894baa36f0f6
  Stored in directory: /root/.cache/pip/wheels/f8/e0/3d/9d0c2020c44a519b9f02ab4fa6d2a4a996c98d79ab2f569fa1
Successfully built sklearn


In [26]:
# General packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds
import seaborn


# Classifiers
from sklearn import model_selection
from sklearn import metrics
from sklearn import feature_selection 
from sklearn import preprocessing
from sklearn import neighbors
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC



In [17]:
# Some functions we will use
def colorplot(clf, ax, x, y, h=100):
    '''
    Overlay the decision areas as colors in an axes.
    
    Input:
        clf: trained classifier
        ax: axis to overlay color mesh on
        x: feature on x-axis
        y: feature on y-axis
        h(optional): steps in the mesh
    '''
    # Create a meshgrid the size of the axis
    xstep = (x.max() - x.min() ) / 20.0
    ystep = (y.max() - y.min() ) / 20.0
    x_min, x_max = x.min() - xstep, x.max() + xstep
    y_min, y_max = y.min() - ystep, y.max() + ystep
    h = max((x_max - x_min, y_max - y_min))/h
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    if hasattr(clf, "decision_function"):
        Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    else:
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    if len(Z.shape) > 1:
        Z = Z[:, 1]
    
    # Put the result into a color plot
    cm = plt.cm.RdBu_r
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)
    del xx, yy, x_min, x_max, y_min, y_max, Z, cm

In [18]:
# Run this to use from colab environment
!git clone https://github.com/jveenland/tm10007_ml.git
import os
import pandas as pd
data = pd.read_csv('/content/tm10007_ml/worcgist/GIST_radiomicFeatures.csv', index_col=0)
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

fatal: destination path 'tm10007_ml' already exists and is not an empty directory.
The number of samples: 246
The number of columns: 494


In [ ]:
#zorgen dat verschillende functies uit de files aangeroepen kunnen worden 
from tm10007_ml.worcgist.load_data import *   #misschien dat dit nog op een nettere manier kan 
dataset = load_data(); 

In [ ]:
#Splitsen in een train en in een testset 

# Classifiers
from sklearn.model_selection import train_test_split

#Waarbij x = features, y = label
#vraag: doet ie dit elke keer op een andere manier? want dan toch lastig conclusies te trekken? --> daarom achter random state een getal - maakt niet uit zolang integer is
y = data['label']
x = data.drop("label", axis='columns')
# df_train, df_test = train_test_split(data, test_size=0.2, random_state=1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=1, stratify=y)
#AFBLIJVEN VAN TESTDATA!! :) 





In [ ]:
#Testen of de data normaal verdeeld is of niet 
from scipy.stats import shapiro
  
# define a function to test for normality using the Shapiro-Wilk test
def test_normality(column):
    stat, p = shapiro(column)
    alpha = 0.05
    if p > alpha:
        return 1
    else:
        return 0

# apply the test_normality function to each column of the DataFrame
normality_results = x_train.apply(test_normality)

# print the results
print(normality_results)
# sum = sum(normality_results)
print(sum)

#dus meeste data is niet normaal verdeeld voor 60 van de gevallen --> dus voor deze wil je 
#een andere schaling gaan toepassen 

# losse dataframes maken 
normalized = pd.DataFrame()
non_normalized = pd.DataFrame()
for ind, elem in zip(x_train.columns, normality_results): 
  if elem == 0: 
    non_normalized = non_normalized.append(x_train[ind])
  else: 
    normalized = normalized.append(x_train[ind])
  

#los scalen 
scaler = preprocessing.RobustScaler()
scaler.fit(non_normalized)
non_normalized = pd.DataFrame(scaler.transform(non_normalized))

scaler_norm = preprocessing.StandardScaler()
scaler_norm.fit(normalized)
normalized = pd.DataFrame(scaler_norm.transform(normalized))

#non_normalized en normalized gecombineerd in een nieuwe dataframe
x_train_scaled = pd.concat([non_normalized, normalized])
scaler_tot = preprocessing.MinMaxScaler()
scaler_tot.fit(x_train_scaled)
x_train_scaled = pd.DataFrame(scaler_tot.transform(x_train_scaled))

#x_train_scaled = normalized.append(non_normalized)
# 1. losse df voor normalized en non normalized 
# 2. scaling met Robust en Standard 
# 3. Samenvoegen en alles met Min max normalizeren 


In [ ]:
# print(non_normalized)

In [ ]:
#plotten 

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1)
print(type(x_train['PREDICT_original_hf_min']))
plt.scatter(x_train['PREDICT_original_phasef_phasesym_peak_position_WL3_N5'], x_train['PREDICT_original_phasef_phasesym_range_WL3_N5'])
# plt.show()

#Testen met een andere variabele 
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1)
print(type(x_train['PREDICT_original_hf_min']))
plt.scatter(x_train['PREDICT_original_sf_convexity_avg_2.5D'], x_train['PREDICT_original_phasef_phasesym_range_WL3_N5'])
# plt.show()

print(x_train)

In [19]:
#zoeken van NaN --> want duidelijk dat deze erin zitten - ook voor de nu geplotte in de eerste figuur 


np.where(np.asanyarray(np.isnan(x_train))) 

# # Find all entries that are not a number
x_train = x_train.apply(lambda x: pd.to_numeric(x, errors='coerce'))
nan_b = pd.isna(x_train)
n_nan = nan_b.sum().sum()
if n_nan > 0:
  nan = np.where(nan_b == "True")
else:
  nan = 0
# # Replace all NaN with 0
# x_train_nieuw = x_train.fillna(0)
print(n_nan)

# volgens mij hebben wij dus alleen rijen waarbij er opeens 0'en zijn, maar 
# hebben wij geen Nan 
#op deze manier worden in x_train alle kolommen gevonden waarbij er alleen maar 
#0'en zijn, dus deze droppen - dit zijn 6 kolommen 
df1 = x_train.mask(x_train != 0).dropna(axis=1)
# print(df1)

#zorgen dat deze rijen verwijderd worden ? willen we dit ?  NOG NIET GELUKT :)
# column_names = list(df1.columns.values)
# df2 = x_train.drop('index', inplace=True, axis=1)

df1 = x_train.mask(x_train != 0).dropna(axis=1)

df2 = x_train.copy()
for col in df1:
    df2 = df2.drop(col, axis=1)


0


In [ ]:
# Attempt at PCA

def pca_func(X,n):
  # create PCA object with 2 components
  pca = PCA(n_components=n)

  # fit and transform the data
  df_pca = pca.fit_transform(X)
  return df_pca

# Hier kan je dus misschien een loopje van maken om te kijken wat het best werkt
n = 100
df_pca_test = pca_func(df2,n)

In [20]:
# SelectKBest 

def selectkbest_func(X,k):
    # select the top n features using chi-squared test
    select = SelectKBest(score_func=chi2, n_features=k)

    # fit and transform the data
    df_selectkbest = select.fit_transform(X)
    return df_selectkbest

In [21]:
# RFE

def rfe_func(X,k):
    # linear regression
    lr = LinearRegression()

    # Create an RFE object and select the top n features (we moeten misschien nog even letten op de step)
    rfe = RFE(estimator=lr, n_features=k, step=1)
  
    #fit and transform the data
    df_rfe = rfe.fit_transform(X)
    return df_rfe

In [31]:
# L1 regularization (Lasso regression)

def lasso_func(X,c): 
    #lasso regression
    lasso = Lasso(alpha=c)

    #fit and transform the data
    df_lasso = lasso.fit_transform(X)
    return df_lasso

In [32]:
# Comparing different feature selection methods

methods = [("f_classif_vs_SelectKBest", SelectKBest(f_classif)),
           ("mutual_info_vs_SelectKBest", SelectKBest(mutual_info_classif)),
           ("lasso_vs_SelectKBest", SelectKBest(Lasso)),
           ("f_classif_vs_lasso", df_lasso(f_classif)),
           ("mutual_info_vs_lasso", Lasso(mutual_info_classif)),]

# ("f_classif_vs_mutual_info", mutual_info_classif(f_classif)),

# Loop over the feature selection methods
for name, method in methods:
    
    # Fit the feature selection method on the training data
    X_train_new = method.fit_transform(x_train, y_train)
    
    # Get the indices of the selected features
    selected_features = method.get_support(indices=True)
    
    # Print the selected features
    print("Selected features using", name, ":", x.columns[selected_features])
    
    # Define the classifiers to compare
    classifiers = [("Random Forest", RandomForestClassifier(random_state=42)),
                   ("Gradient Boosting", GradientBoostingClassifier(random_state=42)),
                   ("SVM", SVC(random_state=42)),
                   ("KNN", KNeighborsClassifier())]
    
    # Loop over the classifiers
    for clf_name, clf in classifiers:
        
        # Train the classifier on the selected features and evaluate its performance
        clf.fit(X_train_new, y_train)
        cv_score = cross_val_score(clf, X_train_new, y_train, cv=5)
        print("Cross-validation accuracy using", clf_name, ":", np.mean(cv_score))
        
        # Transform the test data using the same feature selection method
        #X_test_new = method.transform(X_test)
        
        # Evaluate the performance of the classifier on the test data
        #test_score = clf.score(X_test_new, y_test)
        #print("Test accuracy using", clf_name, ":", test_score)


NameError: ignored